<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>

## [Implementing Neural Networks with Numpy](https://github.com/SurajDonthi/Neural-Networks-from-Scratch)

### [Part 5: Neural Networks from Scratch (with classes)](https://github.com/OmarAflak/Medium-Python-Neural-Network)  

In this post we will go through the mathematics of machine learning and code from scratch, in Python, a small library to build neural networks with a variety of layers (Fully Connected, Convolutional, etc.). Eventually, we will be able to create networks in a modular fashion:

#### Layer by Layer

We need to keep in mind the big picture here :

+ We feed input data into the neural network.
+ The data flows from layer to layer until we have the output.
+ Once we have the output, we can calculate the error which is a scalar.
+ Finally we can adjust a given parameter (weight or bias) by subtracting the derivative of the error with respect to the parameter itself.
    
We iterate through that process.

The most important step is the 4th. We want to be able to have as many layers as we want, and of any type. But if we modify/add/remove one layer from the network, the output of the network is going to change, which is going to change the error, which is going to change the derivative of the error with respect to the parameters. We need to be able to compute the derivatives regardless of the network architecture, regardless of the activation functions, regardless of the loss we use.

In order to achieve that, we must implement each layer separately.

### What every layer should implement

Every layer that we might create (fully connected, convolutional, maxpooling, dropout, etc.) have at least 2 things in common: input and output data.

![Layers](./Images/1nn.png)

### Forward propagation

We can already emphasize one important point which is: the output of one layer is the input of the next one.

![Layers](./Images/2nn.png)

This is called forward propagation. Essentially, we give the input data to the first layer, then the output of every layer becomes the input of the next layer until we reach the end of the network. By comparing the result of the network (Y) with the desired output (let’s say Y*), we can calculate en error E. The goal is to minimize that error by changing the parameters in the network. That is backward propagation (backpropagation).

### Gradient Descent

This is a quick reminder, if you need to learn more about gradient descent there are tons of resources on the internet.

Basically, we want to change some parameter in the network (call it w) so that the total error E decreases. There is a clever way to do it (not randomly) which is the following :

![Layers](./Images/3nn.png)

Where α is a parameter in the range [0,1] that we set and that is called the learning rate. Anyway, the important thing here is ∂E/∂w (the derivative of E with respect to w). We need to be able to find the value of that expression for any parameter of the network regardless of its architecture.
Backward propagation

Suppose that we give a layer the derivative of the error with respect to its output (∂E/∂Y), then it must be able to provide the derivative of the error with respect to its input (∂E/∂X).

![Layers](./Images/4nn.png)

Remember that E is a scalar (a number) and X and Y are matrices.

![Layers](./Images/5nn.png)

Let’s forget about ∂E/∂X for now. The trick here, is that if we have access to ∂E/∂Y we can very easily calculate ∂E/∂W (if the layer has any trainable parameters) without knowing anything about the network architecture! 
We simply use the chain rule :

![Layers](./Images/6nn.png)

The unknown is $∂y_j/∂w$ which totally depends on how the layer is computing its output. So if every layer have access to ∂E/∂Y, where Y is its own output, then we can update our parameters!

### But why ∂E/∂X ?

Don’t forget, the output of one layer is the input of the next layer. Which means ∂E/∂X for one layer is ∂E/∂Y for the previous layer!  

That’s it ! It’s just a clever way to propagate the error ! Again, we can use the chain rule :

![Layers](./Images/7nn.png)

This is very important, it’s the key to understand backpropagation ! After that, we’ll be able to code a Deep Convolutional Neural Network from scratch in no time !
Diagram to understand backpropagation

This is what I described earlier. Layer 3 is going to update its parameters using ∂E/∂Y, and is then going to pass ∂E/∂H2 to the previous layer, which is its own “∂E/∂Y”. Layer 2 is then going to do the same, and so on and so forth.

![Layers](./Images/8nn.png)

This may seem abstract here, but it will get very clear when we will apply this to a specific type of layer. Speaking of abstract, now is a good time to write our first python class.

### Abstract Base Class : Layer

The abstract class Layer, which all other layers will inherit from, handles simple properties which are an input, an output, and both a forward and backward methods.

In [ ]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

As you can see there is an extra parameter in backward_propagation that I didn’t mention, it is the learning_rate. This parameter should be something like an update policy, or an optimizer as they call it in Keras, but for the sake of simplicity we’re simply going to pass a learning rate and update our parameters using gradient descent.

### Fully Connected Layer

Now let's define and implement the first type of layer: fully connected layer or FC layer. FC layers are the most basic layers as every input neurons are connected to every output neurons.

![Layers](./Images/9nn.png)

### Forward Propagation

The value of each output neuron can be calculated as the following :

![Layers](./Images/10nn.png)

With matrices, we can compute this formula for every output neuron in one shot using a dot product:  
![Layers](./Images/11nn.png)

![Layers](./Images/12nn.png)

We’re done with the forward pass. Now let’s do the backward pass of the FC layer.

Note that I’m not using any activation function yet, that’s because we will implement it in a separate layer!
Backward Propagation

As we said, suppose we have a matrix containing the derivative of the error with respect to that layer’s output (∂E/∂Y). We need :

+ The derivative of the error with respect to the parameters (∂E/∂W, ∂E/∂B)
+ The derivative of the error with respect to the input (∂E/∂X)

Let's calculate ∂E/∂W. This matrix should be the same size as W itself : ixj where i is the number of input neurons and j the number of output neurons.  
We need one gradient for every weight:  

![Layers](./Images/13nn.png)

Using the chain rule stated earlier, we can write:

![Layers](./Images/14nn.png)

Therefore,  
![Layers](./Images/15nn.png)

That’s it we have the first formula to update the weights! Now let's calculate ∂E/∂B.  
![Layers](./Images/16nn.png)

Again ∂E/∂B needs to be of the same size as B itself, one gradient per bias. We can use the chain rule again:

![Layers](./Images/17nn.png)

And conclude that:

![Layers](./Images/18nn.png)

Now that we have ∂E/∂W and ∂E/∂B, we are left with ∂E/∂X which is very important as it will “act” as ∂E/∂Y for the layer before that one.

![Layers](./Images/19nn.png)

Again, using the chain rule:  
![Layers](./Images/20nn.png)

Finally, we can write the whole matrix:

![Layers](./Images/21nn.png)

That’s it! We have the three formulas we needed for the FC layer!

![Layers](./Images/22nn.png)

Coding the Fully Connected Layer

We can now write some python code to bring this math to life!

In [ ]:
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### Activation Layer

All the calculation we did until now were completely linear. It's hopeless to learn anything with that kind of model. We need to add non-linearity to the model by applying non-linear functions to the output of some layers.

Now we need to redo the whole process for this new type of layer!

No worries, it’s going to be way faster as there are no learnable parameters. We just need to calculate ∂E/∂X.

We will call f and f' the activation function and its derivative respectively.

![Layers](./Images/23nn.png)

### Forward Propagation

As you will see, it is quite straightforward. For a given input X , the output is simply the activation function applied to every element of X . Which means input and output have the same dimensions.

![Layers](./Images/24nn.png)

### Backward Propagation

Given ∂E/∂Y, we want to calculate ∂E/∂X.

![Layers](./Images/25nn.png)

Be careful, here we are using an element-wise multiplication between the two matrices (whereas in the formulas above, it was a dot product).

### Coding the Activation Layer

The code for the activation layer is as straightforward.

In [ ]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

You can also write some activation functions and their derivatives in a separate file. These will be used later to create an ActivationLayer.

In [4]:
# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

### Loss Function

Until now, for a given layer, we supposed that ∂E/∂Y was given (by the next layer). But what happens to the last layer? How does it get ∂E/∂Y? We simply give it manually, and it depends on how we define the error.

The error of the network, which measures how good or bad the network did for a given input data, is defined by you. There are many ways to define the error, and one of the most known is called MSE — Mean Squared Error.

![Layers](./Images/26nn.png)

Where y* and y denotes desired output and actual output respectively. You can think of the loss as a last layer which takes all the output neurons and squashes them into one single neuron. What we need now, as for every other layer, is to define ∂E/∂Y. Except now, we finally reached E!

![Layers](./Images/27nn.png)

These are simply two python functions that you can put in a separate file. They will be used when creating the network.

In [5]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

Network Class

Almost done ! We are going to make a Network class to create neural networks very easily akin the first picture !

I commented almost every part of the code, it shouldn’t be too complicated to understand if you grasped the previous steps. Nevertheless, leave a comment if you have any question, I will gladly answer !

In [6]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            if i%10==0:
                print('epoch %d/%d   error=%f' % (i+1, epochs, err))

### Building Neural Networks

Finally ! We can use our class to create a neural network with as many layers as we want ! We are going to build two neural networks : a simple XOR and a MNIST solver.

### Solve XOR

Starting with XOR is always important as it’s a simple way to tell if the network is learning anything at all.

In [7]:
### training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.302605
epoch 11/1000   error=0.290115
epoch 21/1000   error=0.287136
epoch 31/1000   error=0.283559
epoch 41/1000   error=0.277893
epoch 51/1000   error=0.268779
epoch 61/1000   error=0.255445
epoch 71/1000   error=0.239371
epoch 81/1000   error=0.224699
epoch 91/1000   error=0.214088
epoch 101/1000   error=0.206673
epoch 111/1000   error=0.200699
epoch 121/1000   error=0.194872
epoch 131/1000   error=0.188219
epoch 141/1000   error=0.179592
epoch 151/1000   error=0.167088
epoch 161/1000   error=0.147598
epoch 171/1000   error=0.117250
epoch 181/1000   error=0.076374
epoch 191/1000   error=0.040932
epoch 201/1000   error=0.022581
epoch 211/1000   error=0.014220
epoch 221/1000   error=0.009960
epoch 231/1000   error=0.007501
epoch 241/1000   error=0.005939
epoch 251/1000   error=0.004875
epoch 261/1000   error=0.004110
epoch 271/1000   error=0.003537
epoch 281/1000   error=0.003094
epoch 291/1000   error=0.002743
epoch 301/1000   error=0.002459
epoch 311/1000   er

Clearly this is working, great ! We can now solve something more interesting, let’s solve MNIST!

### Solve MNIST

We didn’t implemented the Convolutional Layer but this is not a problem. All we need to do is to reshape our data so that it can fit into a Fully Connected Layer.

MNIST Dataset consists of images of digits from 0 to 9, of shape 28x28x1. The goal is to predict what digit is drawn on a picture.

In [8]:
from keras.datasets import mnist
from keras.utils import np_utils

In [9]:
# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.240671
epoch 11/35   error=0.025668
epoch 21/35   error=0.012187
epoch 31/35   error=0.006614


predicted values : 
[array([[-0.0160132 ,  0.01509453,  0.01006175, -0.02688088, -0.07208044,
        -0.01824391,  0.01822099,  0.97089147,  0.00535009,  0.02148698]]), array([[ 1.84963233e-03, -1.23253018e-03,  4.38965024e-01,
         4.26556539e-02,  4.66184276e-02,  2.48904238e-01,
         7.13771284e-02, -1.32382095e-04,  2.69515446e-01,
        -3.82109553e-02]]), array([[ 0.08314035,  0.97151282,  0.08111742,  0.06695823, -0.07338969,
        -0.05588953, -0.00130393, -0.04862157, -0.08132707, -0.04341171]])]
true values : 
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


This is working perfectly ! Amazing :)